In [12]:
import tensorflow as tf

In [13]:
text_ds = tf.data.TextLineDataset("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/manual_dataset/FILTERED_DATASET.txt").filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [14]:
codelines = []
with open('FILTERED_DATASET.txt') as obj:
    codelines = obj.readlines()

codelines

['#include <stdio.h>\n',
 '\n',
 'int main(void)\n',
 '{\n',
 '    int result = 1;\n',
 '    if (result == 1)\n',
 '    {\n',
 '        printf("Executing critical code...\\n");\n',
 '    }\n',
 '    else\n',
 '    {\n',
 '        printf("Exiting out...\\n");\n',
 '        return 1;\n',
 '    }\n',
 '\n',
 '    return 0;\n',
 '}\n']

In [15]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    
    

    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    for line_number in range(len(raw_codeLines)):
        placeHolder = raw_codeLines[line_number]
        placeHolder = space_out(placeHolder, ";")
        placeHolder = space_out(placeHolder, "(")
        placeHolder = space_out(placeHolder, ")")
        placeHolder = space_out(placeHolder, ",")
        if placeHolder.endswith(";"):
            raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        elif not(placeHolder.endswith(";")):
            raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [16]:
codelines = code_preprocessing('FILTERED_DATASET.txt')

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()
output = vectorizer.fit_transform(codelines)
output

<18x17 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

In [19]:
inverse_vocab = {index: token for token, index in vectorizer.vocabulary_.items()}
for i in (inverse_vocab):
    print(inverse_vocab.get(i),':', i)

start : 14
include : 7
stdio : 15
end : 3
int : 8
main : 9
void : 16
result : 12
if : 6
printf : 11
executing : 4
critical : 1
code : 0
else : 2
exiting : 5
out : 10
return : 13


In [20]:
for i in enumerate(output):
    print(codelines[i[0]])
    print()
    print(i[1].A[:1])
    print()
    
    

<start> #include <stdio.h> <end>

[[0.         0.         0.         0.20787462 0.         0.
  0.         0.67586104 0.         0.         0.         0.
  0.         0.         0.20787462 0.67586104 0.        ]]

<start>  <end>

[[0.         0.         0.         0.70710678 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.70710678 0.         0.        ]]

<start> int main ( void )  <end>

[[0.         0.         0.         0.17891256 0.         0.
  0.         0.         0.50915414 0.58169694 0.         0.
  0.         0.         0.17891256 0.         0.58169694]]

<start> { <end>

[[0.         0.         0.         0.70710678 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.70710678 0.         0.        ]]

<start>     int result = 1 ;  <end>

[[0.         0.         0.         0.23441998 0.         0.
  0.         0.         0.66711863 0.         0.         0.
  0.66711863 0.       